# Breeze API

In [1]:
import requests
import json

url = "https://nlplab-llm.nlpnchu.org/generate"

headers = {
  'Content-Type': 'application/json',
  'accept': 'application/json'
}

def get_breeze(input_content):
    system = 'You are a helpful AI assistant built by MediaTek Research. The user you are helping speaks Traditional Chinese and comes from Taiwan.'

    payload = json.dumps({
        "inputs": f"<s> {system} [INST] {input_content} [/INST]",
        "parameters": {
            "do_sample": True,
            "temperature": 0.01,
            "top_p": 0.95,
            # "frequency_penalty": 0.01,
            # "presence_penalty": 0.05,
            'repetition_penalty': 1, # 不要一直生重複的
            'max_new_tokens':1000, # 要設一下，不然會講到一半就停掉
        }
        })
    headers = {
    'Content-Type': 'application/json',
    'accept': 'application/json'
    }
    try:
        response = requests.request("POST", url, headers=headers, data=payload)
        j_result=json.loads(response.text)
        if "generated_text" in j_result.keys():
            return j_result['generated_text']
        else:
            return j_result
    except Exception as e:
        print(e)

## 做情節摘要
* 測試不同長度做摘要的效果
* 5k以下可以直接做
* 5k以上，用sliding window切，window size=4000，overlap=100，先做一次摘要。每個window做完後，再做全部的情節摘要

In [2]:
import json

with open('dataset/train_data.json', 'r', encoding='utf-8') as f:
    train_data = json.load(f)

In [3]:
for idx, data in enumerate(train_data[:10]):
    print(idx, len(data['fact']))

0 6546
1 3385
2 1331
3 10172
4 9266
5 13669
6 328
7 7083
8 8806
9 12726


## 目標：<10k的事實，看給不給情節摘要範例效果如何
* 結果：3k的事實，給情節摘要範例效果最好

In [4]:
def with_example(judgement):
    example = 'exaple'    
    summary_prompt = f'我會給你一篇法律判決書內容，幫我做情節摘要，可能包含事實情節描述、判決結果及理由。{example}判決書內容：輸入：{judgement}。輸出：'
    summary = get_breeze(summary_prompt)
    print(f'='*100)
    print(f'有範例的情節摘要')
    return summary

def without_example(judgement):
    summary_prompt = f'我會給你一篇法律判決書內容，幫我做情節摘要，可能包含事實情節描述、判決結果及理由。判決書內容如下：{judgement}。'
    summary = get_breeze(summary_prompt)
    print(f'='*100)
    print(f'沒有範例的情節摘要')
    return summary

### 長度1331

In [16]:
judgement = train_data[2]['fact']
print(f'Length: {len(judgement)}')
scenario_with_example = with_example(judgement)
print(scenario_with_example)
scenario_without_example = without_example(judgement)
print(scenario_without_example)

Length: 1331
有範例的情節摘要
 本案件涉及偽造婚姻以使泰國籍女子白雪蘭得以在臺灣地區工作。被告白秋霖與白雪蘭明知並無結婚真意，於2004年在泰國辦理假結婚手續，並在臺灣辦理結婚登記。之後，被告於2005年訴請離婚，再次辦理離婚登記。白雪蘭因逾期居留於2009年被查獲。被告的行為涉及使公務員登載不實文書，分別觸犯刑法第216條和第214條。在協商程序中，被告同意受有期徒刑6月，減為3月，緩刑2年之宣告。
沒有範例的情節摘要
 情節摘要：

被告白秋霖與泰國籍女子白雪蘭（WANGMONTRIDOKMAI）共同明知沒有結婚的真意，但為了使白雪蘭得以結婚方式在臺灣地區工作，兩人在泰國辦理假結婚手續，取得不實的結婚證明書等資料。白秋霖於93年11月18日持這些文件前往臺中縣沙鹿鎮戶政事務所，申請辦理結婚戶籍登記，使承辦人員將虛偽辦理結婚的不實事項登載於職務上掌管的戶籍登記資料電子資訊檔案紀錄和戶籍謄本等公文書，損害戶政機關對於戶籍登記管理的正確性。

在94年3月30日，兩人向法院訴請離婚，經法院判決離婚後，白秋霖於94年7月7日再次前往戶政事務所辦理離婚登記，使承辦人員將虛偽辦理離婚的不實事項登載於戶籍登記資料。最後，白雪蘭因逾期居留於99年4月6日被查獲。

判決結果：被告白秋霖與白雪蘭達成協商合意，被告願受有期徒刑6月，減為有期徒刑3月，緩刑2年之宣告。


### 長度3385

In [17]:
judgement = train_data[1]['fact']
print(f'Length: {len(judgement)}')
scenario_with_example = with_example(judgement)
print(scenario_with_example)
scenario_without_example = without_example(judgement)
print(scenario_without_example)

Length: 3385
有範例的情節摘要
 本案涉及偽造私文書和行使偽造私文書等犯罪。被告甲○○在未經許可的情況下，偽造奇美醫院、院長邱仲慶和醫師甲○○的印章，並製作了一份診斷證明書，意圖欺騙法院。被告向法院提交了這個偽造的診斷證明書，以支持其主張因照顧住院的母親而延遲報繳槍枝的辯解。法院發現診斷證明書上的內容與被告的母親實際就診情況不符，因此認定被告偽造私文書並行使偽造私文書。被告在5年內故意再犯有期徒刑以上的罪，因此應依法加重其刑。法院判處被告有期徒刑，並沒收偽造的印章和印文。
沒有範例的情節摘要
 1. 情節摘要：

本案涉及被告甲○○於民國100年11月1日拾得可發射子彈具有殺傷力之槍枝後持有，並於100年11月6日向員警坦承犯行。被告為掩飾其持有槍枝之犯意，在100年11月6日至101年1月12日間偽造奇美醫療財團法人奇美醫院（下稱奇美醫院）、院長邱仲慶及醫師甲○○之印章，並偽造如附表編號四所示診斷證明書（系爭診斷證明書），向法院提出以表示系爭診斷證明書所載內容經認證屬實。被告此偽造私文書之行為足以生損害於奇美醫院、邱仲慶、甲○○及國家司法權行使之正確性。

2. 判決結果及理由：

被告甲○○被依刑法第216條、第210條之行使偽造私文書罪判刑。被告偽造印章及印文均屬偽造私文書之部分行為，偽造私文書之行為應為行使偽造私文書之高度行為所吸收，均不另論罪。被告為累犯，應依刑法第47條第1項加重其刑。

3. 判決結果：

被告甲○○被判處有期徒刑1年6月。偽造如附表編號一至編號三所示之印章雖未扣案，但無證據證明已滅失，另偽造如附表編號四所示「奇美醫療財團法人奇美醫院證明用章」、「奇美醫院院長邱仲慶（乙）」、「急診醫學科甲○○診斷書用」之印文，不問屬於犯人與否，均依刑法第219條之規定宣告沒收。系爭診斷證明書經被告提出於法院而行使，非屬被告所有之物，爰不予宣告沒收。


### 長度6546

In [18]:
judgement = train_data[0]['fact']
print(f'Length: {len(judgement)}')
scenario_with_example = with_example(judgement)
print(scenario_with_example)
scenario_without_example = without_example(judgement)
print(scenario_without_example)

Length: 6546
有範例的情節摘要
 1. 情節摘要：
2. 被告陳泰龍與陳玉婉（泰國籍）假結婚，以便陳玉婉來臺居留工作。
3. 被告等人共同偽造文書，使相關機關人員陷於錯誤，核發居留證予陳玉婉。
4. 被告否認犯行，未坦承犯行。
5. 被告適用修正前刑法，較有利。
6. 被告共同行使使公務員登載不實文書罪，處有期徒刑1年。
7. 被告共同行使偽造變造或登載不實之文書罪，處有期徒刑1年。
8. 被告共同行使使公務員登載不實文書罪，處有期徒刑1年。
9. 被告共同行使偽造變造或登載不實之文書罪，處有期徒刑1年。
10. 被告共同行使使公務員登載不實文書罪，處有期徒刑1年。
11. 被告共同行使偽造變造或登載不實之文書罪，處有期徒刑1年。
12. 被告共同行使使公務員登載不實文書罪，處有期徒刑1年。
13. 被告共同行使偽造變造或登載不實之文書罪，處有期徒刑1年。
14. 被告共同行使使公務員登載不實文書罪，處有期徒刑1年。
15. 被告共同行使偽造變造或登載不實之文書罪，處有期徒刑1年。
16. 被告共同行使使公務員登載不實文書罪，處有期徒刑1年。
17. 被告共同行使偽造變造或登載不實之文書罪，處有期徒刑1年。
18. 被告共同行使使公務員登載不實文書罪，處有期徒刑1年。
19. 被告共同行使偽造變造或登載不實之文書罪，處有期徒刑1年。
20. 被告共同行使使公務員登載不實文書罪，處有期徒刑1年。
21. 被告共同行使偽造變造或登載不實之文書罪，處有期徒刑1年。
22. 被告共同行使使公務員登載不實文書罪，處有期徒刑1年。
23. 被告共同行使偽造變造或登載不實之文書罪，處有期徒刑1年。
24. 被告共同行使使公務員登載不實文書罪，處有期徒刑1年。
25. 被告共同行使偽造變造或登載不實之文書罪，處有期徒刑1年。
26. 被告共同行使使公務員登載不實文書罪，處有期徒刑1年。
27. 被告共同行使偽造變造或登載不實之文書罪，處有期徒刑1年。
28. 被告共同行使使公務員登載不實文書罪，處有期徒刑1年。
29. 被告共同行使偽造變造或登載不實之文書罪，處有期徒刑1年。
30. 被告共同行使使公務員登載不實文書罪，處有期徒刑1年。
31. 被告共同行使偽造變造或登載不實之文書罪，處有期徒刑1年。
32. 被告共同行使使公務員登載不實文書罪，處有期徒刑1年。
33.

### 長度9266

In [19]:
judgement = train_data[4]['fact']
print(f'Length: {len(judgement)}')
scenario_with_example = with_example(judgement)
print(scenario_with_example)
scenario_without_example = without_example(judgement)
print(scenario_without_example)

Length: 9266
有範例的情節摘要
 1. 被告謝麗華涉嫌違法登載在苗成公司102年11月6日股東名冊之行為，顯係屬被告所犯，被告所犯行登載不實之行為，足以生損害於公眾或告訴人，故被告應受法律處罰。2. 被告謝麗華涉嫌違法登載在苗成公司102年11月6日股東名冊之行為，顯係屬被告所犯行，故被告應受法律處罰。3. 被告謝麗華涉嫌違法登載在苗成公司102年11月6日股東名冊之行為，顯係屬被告所犯行，故被告應受法律處罰。4. 被告謝麗華涉嫌違法登載在苗成公司102年11月6日股東名冊之行為，顯係屬被告所犯行，故被告應受法律處罰。5. 被告謝麗華涉嫌違法登載在苗成公司102年11月6日股東名冊之行為，顯係屬被告所犯行，故被告應受法律處罰。6. 被告謝麗華涉嫌違法登載在苗成公司102年11月6日股東名冊之行為，顯係屬被告所犯行，故被告應受法律處罰。7. 被告謝麗華涉嫌違法登載在苗成公司102年11月6日股東名冊之行為，顯係屬被告所犯行，故被告應受法律處罰。8. 被告謝麗華涉嫌違法登載在苗成公司102年11月6日股東名冊之行為，顯係屬被告所犯行，故被告應受法律處罰。9. 被告謝麗華涉嫌違法登載在苗成公司102年11月6日股東名冊之行為，顯係屬被告所犯行，故被告應受法律處罰。10. 被告謝麗華涉嫌違法登載在苗成公司102年11月6日股東清冊之行為，顯係屬被告所犯行，故被告應受法律處罰。11. 被告謝麗華涉嫌違法登載在苗成公司102年11月6日股東清冊之行為，顯係屬被告所犯行，故被告應受法律處罰。12. 被告謝麗華涉嫌違法登載在苗成公司102年11月6日股東清冊之行為，顯係屬被告所犯行，故被告應受法律處罰。13. 被告謝麗華涉嫌違法登載在苗成公司102年11月6日股東清冊之行為，顯係屬被告所犯行，故被告應受法律處罰。14. 被告謝麗華涉嫌違法登載在苗成公司102年11月6日股東清冊之行為，顯係屬被告所犯行，故被告應受法律處罰。15. 被告謝麗華涉嫌違法登載在苗成公司102年11月6日股東清冊之行為，顯係屬被告所犯行，故被告應受法律處罰。16. 被告謝麗華涉嫌違法登載在苗成公司102年11月6日股東清冊之行為，顯係屬被告所犯行，故被告應受法律處罰。17. 被告謝麗華涉嫌違法登載在苗成公司102年11月6日股東清冊之行為，顯係屬被告所犯行，故被告應受法律處罰。18. 被告謝麗華

## 目標：把超過3k的事實，先做事實內容摘要。壓到3k個字以下，再做情節摘要加給範例
* 結果：

### function

In [ ]:
def with_example(judgement):
    example = '以下是一個情節摘要的範例，輸入是判決書內容，輸出是情節摘要。範例如下：輸入:孫桂蘭與孟奇於民國83年7月10日結婚,孫桂蘭係孟奇之子女即孟慶明、孟慶虹、孟慶經、孟慶緯、孟蓉之繼母。孟奇於96年1月22日死亡,孫桂蘭明知孟奇死亡後,其名下財產均為遺產,於遺產分割前屬孫桂蘭、孟慶明、孟慶虹、孟慶經、孟慶緯、孟蓉全體繼承人公同共有,未經全體繼承人同意,任一繼承人不得擅自處分,竟意圖為自己不法所有,基於詐欺取財及行使偽造私文書之犯意,於附表各編號所示之時間,至附表各編號所示金融機構,冒用孟奇之名義,在空白取款憑證填寫如附表各編號所示之日期、帳號及金額,並於「取款印鑑」欄,使用孟奇之印章盜蓋印文1枚,表示係孟奇欲自附表各編號所示金融機構帳戶內提領附表各編號所示提款金額,而偽造孟奇名義之取款憑證私文書後,再連同上開帳戶存摺持之向不知情之附表各編號所示金融機構承辦人員辦理取款而行使之,致該金融機構承辦人員陷於錯誤,誤認孫桂蘭係經孟奇本人授權而為上開手續,如數交付如附表各編號所示金額,均足以生損害於孟慶明、孟慶虹、孟慶經、孟慶緯(已歿)、孟蓉之權益及日盛國際商業銀行股份有限公司《下稱日盛商業銀行》、合作金庫商業銀行股份有限公司竹塹分行《下稱合作金庫商業銀行竹塹分行》、中華郵政股份有限公司清華大學郵局《下稱清華大學郵局》、永豐商業銀行股份有限公司《下稱永豐商業銀行》等金融機構對於孟奇帳戶資料、存款管理之正確性。 輸出:在這個案件中,涉及到一段婚姻關係,並有繼承人之間的財產分割。一名已故的個人的遺產成為了糾紛的焦點。在其去世後,其配偶及子女成為了繼承人。然而,配偶明知其去世,卻企圖以詐欺和偽造私文書的方式來獲取他們的共同財產。她冒用已故人的名義,填寫了取款憑證並偽造了他的印章,使金融機構誤以為是合法的取款要求。最終,金融機構執行了這些虛偽的指示,導致繼承人之間的權益受損,也對涉及的金融機構造成了損害。这些行為涉及偽造私文書、詐欺取財,並對繼承人和金融機構產生了不良影響。請學習以上範例，將以下判決書內容做情節摘要，'
    summary_prompt = f'我會給你一篇法律判決書內容，幫我做情節摘要，可能包含事實情節描述、判決結果及理由。{example}判決書內容：輸入：{judgement}。輸出：'
    summary = get_breeze(summary_prompt)
    print(f'='*100)
    print(f'有範例的情節摘要')
    return summary

def without_example(judgement):
    summary_prompt = f'我會給你一篇法律判決書內容，幫我做情節摘要，可能包含事實情節描述、判決結果及理由。判決書內容如下：{judgement}。'
    summary = get_breeze(summary_prompt)
    print(f'='*100)
    print(f'沒有範例的情節摘要')
    return summary

In [57]:
def split_passage(data):
    summaries = []

    data_len = len(data)
    passage_len = 3000
    overlap_size = 100

    start = 0
    end = min(passage_len, data_len)

    while start < data_len:
        passage = data[start:end]
        # print(f'passage: \n{passage}')
        fact_summary_prompt = f'我會給你一篇法律判決書內容，幫我做事實摘要，可能包含事實情節描述。判決書內容如下：{passage}。'
        summary = get_breeze(fact_summary_prompt)
        # print(f'summary: \n{summary}')
        summaries.append(summary)
        
        start += passage_len - overlap_size
        end = min(start + passage_len, data_len)

    summary_str = ''.join(summaries)

    return summary_str

### 長度6546

In [58]:
judgement = train_data[0]['fact']
print(f'Origin Length: {len(judgement)}')

if len(judgement) > 3000:
    fact_summary = split_passage(judgement)

print(f'Length: {len(fact_summary)}')
print(f'First Summayr: {fact_summary}')
print(f'='*100)
scenario_with_example = with_example(fact_summary)
print(scenario_with_example)
scenario_without_example = without_example(fact_summary)
print(scenario_without_example)

Origin Length: 6546
Length: 905
First Summayr:  判決書摘要：
被告陳泰龍明知與泰國籍女子陳玉婉（泰國名字為JARUWANWONGPRATOOM）間並無結婚之真意，仍與林清水、詹巧薇、張晉源等人共同基於行使使公務員登載不實公文書之概括犯意聯絡，辦理假結婚。被告依林清水於報紙上刊登之徵人廣告應徵，由林清水告知泰國女子欲以假結婚之方式申請來臺居留工作，其若擔任假結婚之人頭老公，除不用負擔前往泰國之食宿、機票費用外，尚可獲取5、6萬元之報酬。被告與林清水、詹巧薇、張晉源等人共同辦理假結婚，使公務員陷於錯誤，登載不實事項於戶籍登記資料內，並核發戶籍謄本。被告與陳玉婉辦理假結婚後，陳玉婉來臺後，被告與陳玉婉共同持登載不實之戶籍謄本等資料，先後向臺南縣政府警察局外事課申請核發陳玉婉之外僑居留證及申請居留證延期，使主管機關對外僑居留管理之正確性受損。 事實摘要：

蔡明峰、林清水、詹巧薇和張晉源涉嫌擔任人頭老公與泰國女子假結婚。蔡明峰因本案被告與共犯陳玉婉辦理假結婚，涉嫌共同行使使公務員登載不實文書，經臺灣臺南地方法院檢察官認為與前開95年度簡上字第330號、99年度簡字第1313號論罪科刑部分有連續犯之裁判上一罪關係，故均以99年度偵字第17571、17771號、100年度偵字第4564號為不起訴處分確定。被告與共犯林清水並無仇怨，被告提供一部機車予共犯林清水使用。被告與共犯陳玉婉確係假結婚，被告的辯解為卸責之詞。被告適用行為時即修正前刑法之相關規定，對其較為有利，自應一體適用修正前刑法規定予以處斷。被告於93年5月19日、93年6月14日及94年5月5日先後多次行使使公務員登載不實文書之犯行，時間密接，所犯係構成要件相同之罪名，顯係基於概括犯意反覆為之，為連續犯，應依修正前刑法第56條規定論以一罪。 事實摘要：被告涉嫌於96年5月28日及96年9月3日與共犯陳玉婉共同持登載不實之戶籍謄本等資料，先後向臺南縣政府警察局外事課申請核發陳玉婉之延期居留等事宜。被告涉嫌共同行使使公務員登載不實文書犯行，均在95年7月1日刑法修正廢除第56條連續犯規定以後，未記載於起訴書犯罪事實內。由於此部分犯行在刑法修正後，未構成連續犯，故非本院得審判之範圍。
有範例的情節摘要
 被告陳泰龍與泰國籍女子陳玉婉（JARUWANWONGPRATOOM）共同辦

### 長度7083

In [59]:
judgement = train_data[7]['fact']
print(f'Origin Length: {len(judgement)}')

if len(judgement) > 3000:
    fact_summary = split_passage(judgement)

print(f'Length: {len(fact_summary)}')
print(f'First Summayr: {fact_summary}')
print(f'='*100)
scenario_with_example = with_example(fact_summary)
print(scenario_with_example)
scenario_without_example = without_example(fact_summary)
print(scenario_without_example)

Origin Length: 7083
Length: 1666
First Summayr:  以下是根據您提供的法律判決書內容所做的事實摘要：

判決書內容摘要：
一、被告乙○○明知其兄甲○○於2014年擔任屏東縣恆春鎮公所清潔隊代理隊長，但恆春鎮清潔隊徵才公告要求錄取人員不得與機關首長或長官有三親等以內的親屬關係。乙○○仍參加徵選，通過面試和體能測驗，並被列為正取。為使公務員登載不實，乙○○出具與首長或主管長官無三親等親屬關係的具結書，並提供相關證件。恆春鎮公所承辦人員丁○○和戊○○形式上審核後，誤以為乙○○符合進用條件，登載乙○○為恆春鎮清潔隊員，表彰其符合進用條件。
二、案件經法務部廉政署移送臺灣屏東地方檢察署檢察官偵查後起訴。

理由：
一、程序部分：
1.被告以外之人於審判外之陳述，經當事人同意作為證據，法院審酌適當者，得為證據。
2.本院引用之供述證據，被告及其辯護人同意有證據能力。
3.本院引用之非供述證據，無證據證明係公務員違背法定程序所取得，且被告及其辯護人未主張排除證據能力，且本院審酌無顯有不可信之情況，得為證據。

二、實體部分：
1.被告乙○○承認在甄選前即知其兄甲○○為清潔隊代理隊長，仍填具具結書，表示與鎮公所首長及主管長官無三親等關係。
2.被告乙○○被錄取後，恆春鎮公所以函文通知錄取，另以函文函請屏東縣政府備查。
3.被告乙○○使公務員登載不實：
a.恆春鎮公所徵才公告要求錄取人員不得與機關首長或長官有三親等以內親屬關係。
b.被告乙○○與長官甲○○為二親等旁系血親，不符合徵才要求。
c.恆春鎮公所承辦人員僅形式上審核具結書，誤以為乙○○符合進用條件，登載不實。
4.鎮公所人員不具實質審核權：
a.證人丁○○、戊○○僅形式上審核具結書，無法確認其真實性。 事實摘要：

1. 被告乙○○明知其與被告甲○○具二親等旁系血親關係，竟仍不實具結，使承辦人誤信被告乙○○符合徵選條件，進而於職務上所掌之本案函文一、二登載被告乙○○因符合條件並受錄取之事項。
2. 被告甲○○於100年5月1日起擔任屏東縣恆春鎮清潔隊代理隊長，明知被告乙○○為其胞妹，仍分別於103年4月23日簽陳恆春鎮公所恆鎮清字第10330712700號函（稿）、103年5月2日簽陳恆春鎮公所恆鎮清字第10330784100號函（稿）時，函稿中核章表示認可為不實之登載，並經鎮長

### 長度9266

In [63]:
judgement = train_data[4]['fact']
print(f'Origin Length: {len(judgement)}')

if len(judgement) > 3000:
    fact_summary = split_passage(judgement)

print(f'Length: {len(fact_summary)}')
print(f'First Summayr: {fact_summary}')
print(f'='*100)
scenario_with_example = with_example(fact_summary)
print(scenario_with_example)
scenario_without_example = without_example(fact_summary)
print(scenario_without_example)

Origin Length: 9266


Length: 1572
First Summayr:  1. 事實摘要：
2013年，苗成公司董事長謝麗華為留任施東益為公司招攬業務，在公司辦理現金增資時，以自己的資金出資以施東益名義繳納300萬元、350萬元現金股款，將增資股份3,000股、3,500股登記在施東益名下，使施東益成為苗成公司股東。
2014年，施東益離職後，謝麗華欲取回施東益的股份，指示不知情的苗成公司員工在財政部國稅局網站填載施東益出售股份給謝麗華的子女和媳婦，並繳納證券交易稅。謝麗華也命令苗成公司員工將繳納完畢的繳款書交給會計事務所，將交易過戶股份的資料登載在苗成公司股東名冊上。
3. 理由：
謝麗華承認上述事實，但否認業務登載不實。辯護人主張謝麗華並非贈與施東益股份，且施東益離職後催促過戶股份。
4. 證據能力：
被告以外之人於審判外的陳述，原則上不得作為證據，但被告以外之人於審判外陳述之傳聞證據，經當事人於審判程序同意作為證據，法院審酌該言詞陳述或書面陳述作成時之情況，認為適當者，亦得為證據。
5. 認定犯罪事實所憑證據：
被告坦承部分事實，證人施東益、黃素絹的證詞，以及苗成公司股東名冊、繳款書等文件。
6. 被告的辯解：
被告辯稱借用施東益名義繳納股款，目的為使施東益對外談生意容易，而非贈與施東益。施東益離職後催促過戶股份。
7. 辯護人的辯護：
辯護人主張被告並非贈與施東益股份，且施東益離職後催促過戶股份。 1. 判決書內容涉及告訴人（證人）於偵查及審理時的具結證述，以及證人黃素絹、張淑燕、柯碧招的證詞，均與事實相符。
2. 苗成公司於99年7月12日、100年7月28日辦理每股1,000元現金增資時，被告以告訴人名義出資繳納300萬元、350萬元現金股款，並在苗成公司股東名冊上記載告訴人因增資而為苗成公司股東，持有3,000股、3,500股。
3. 告訴人於97年1月23日起在苗成公司任職，直至102年8月31日離職。
4. 苗成公司與德律公司的交易額自99年、100年大幅提升，德律公司為上市公司，均有照實付款。
5. 告訴人自97年任職苗成公司後，成功為公司招攬相當程度的業務。
6. 被告為安撫告訴人對其薪資不滿，於99年7月12日、100年7月28日以告訴人名義繳納300萬元、350萬元現金股款，並將苗成公司股份3,000股、3,500股登記在告訴人名下，使告訴人成為苗成公司股

# 整理

In [29]:
def with_example(judgement):
    example = '以下是一個情節摘要的範例，輸入是判決書內容，輸出是情節摘要。範例如下：輸入:孫桂蘭與孟奇於民國83年7月10日結婚,孫桂蘭係孟奇之子女即孟慶明、孟慶虹、孟慶經、孟慶緯、孟蓉之繼母。孟奇於96年1月22日死亡,孫桂蘭明知孟奇死亡後,其名下財產均為遺產,於遺產分割前屬孫桂蘭、孟慶明、孟慶虹、孟慶經、孟慶緯、孟蓉全體繼承人公同共有,未經全體繼承人同意,任一繼承人不得擅自處分,竟意圖為自己不法所有,基於詐欺取財及行使偽造私文書之犯意,於附表各編號所示之時間,至附表各編號所示金融機構,冒用孟奇之名義,在空白取款憑證填寫如附表各編號所示之日期、帳號及金額,並於「取款印鑑」欄,使用孟奇之印章盜蓋印文1枚,表示係孟奇欲自附表各編號所示金融機構帳戶內提領附表各編號所示提款金額,而偽造孟奇名義之取款憑證私文書後,再連同上開帳戶存摺持之向不知情之附表各編號所示金融機構承辦人員辦理取款而行使之,致該金融機構承辦人員陷於錯誤,誤認孫桂蘭係經孟奇本人授權而為上開手續,如數交付如附表各編號所示金額,均足以生損害於孟慶明、孟慶虹、孟慶經、孟慶緯(已歿)、孟蓉之權益及日盛國際商業銀行股份有限公司《下稱日盛商業銀行》、合作金庫商業銀行股份有限公司竹塹分行《下稱合作金庫商業銀行竹塹分行》、中華郵政股份有限公司清華大學郵局《下稱清華大學郵局》、永豐商業銀行股份有限公司《下稱永豐商業銀行》等金融機構對於孟奇帳戶資料、存款管理之正確性。 輸出:在這個案件中,涉及到一段婚姻關係,並有繼承人之間的財產分割。一名已故的個人的遺產成為了糾紛的焦點。在其去世後,其配偶及子女成為了繼承人。然而,配偶明知其去世,卻企圖以詐欺和偽造私文書的方式來獲取他們的共同財產。她冒用已故人的名義,填寫了取款憑證並偽造了他的印章,使金融機構誤以為是合法的取款要求。最終,金融機構執行了這些虛偽的指示,導致繼承人之間的權益受損,也對涉及的金融機構造成了損害。这些行為涉及偽造私文書、詐欺取財,並對繼承人和金融機構產生了不良影響。請學習以上範例，將以下判決書內容做情節摘要，'
    scenario_prompt = f'我會給你一篇法律判決書內容，幫我做情節摘要，可能包含事實情節描述、判決結果及理由。{example}判決書內容：輸入：{judgement}。輸出：'
    scenario = get_breeze(scenario_prompt)
    return scenario

def without_example(judgement):
    scenario_prompt = f'我會給你一篇法律判決書內容，幫我做情節摘要，可能包含事實情節描述、判決結果及理由。判決書內容如下：{judgement}。'
    scenario = get_breeze(scenario_prompt)
    return scenario

def split_passage(judgement):
    summaries = []

    judgement_len = len(judgement)
    window_size = 3000
    overlap_size = 100
    start = 0
    end = min(window_size, judgement_len)

    while start < judgement_len:
        passage = judgement[start:end]
        fact_summary_prompt = f'我會給你一篇法律判決書內容，幫我做事實摘要，可能包含事實情節描述。判決書內容如下：{passage}。'
        summary = get_breeze(fact_summary_prompt)
        summaries.append(summary)
        
        start += window_size - overlap_size
        end = min(start + window_size, judgement_len)

    summary_str = ''.join(summaries)

    return summary_str

def process_judgement(judgement):
    print(f'Origin Length: {len(judgement)}')
    if len(judgement) > 3000:
        print(f'Judgement processing...')
        fact_summary = split_passage(judgement)
        print(f'First summary complete!')
        print(f'Scenario generating...')
        scenario_with_example = with_example(fact_summary)
        scenario_without_example = without_example(fact_summary)
        print(f'Scenario complete!')
        print(f'='*100)
        return {
            "judgement": judgement,
            "fact_summary": fact_summary,
            "with_example": scenario_with_example,
            "without_example": scenario_without_example
        }
    else:
        print(f'Scenario generating...')
        scenario_with_example = with_example(judgement)
        scenario_without_example = without_example(judgement)
        print(f'Scenario complete!')
        print(f'='*100)
        return {
            "judgement": judgement,
            "fact_summary": "None",
            "with_example": scenario_with_example,
            "without_example": scenario_without_example
        }

In [16]:
judgement_1 = train_data[2]['fact']
judgement_2 = train_data[7]['fact']

print("Judgement 1:")
with_example_1, without_example_1 = process_judgement(judgement_1)
print("With Example:")
print(with_example_1)
print("Without Example:")
print(without_example_1)

print("=" * 50)

print("Judgement 2:")
with_example_2, without_example_2 = process_judgement(judgement_2)
print("With Example:")
print(with_example_2)
print("Without Example:")
print(without_example_2)

Judgement 1:
Origin Length: 1331
Skip Split...
With Example:
 本案件涉及偽造婚姻以使泰國籍女子白雪蘭得以在臺灣地區工作。被告白秋霖與白雪蘭明知並無結婚真意，於2004年在泰國辦理假結婚手續，並在臺灣辦理結婚登記。之後，被告於2005年訴請離婚，再次辦理離婚登記。白雪蘭因逾期居留於2009年被查獲。被告的行為涉及使公務員登載不實文書，分別觸犯刑法第216條和第214條。在協商程序中，被告同意受有期徒刑6月，減為3月，緩刑2年之宣告。
Without Example:
 情節摘要：

被告白秋霖與泰國籍女子白雪蘭（WANGMONTRIDOKMAI）共同明知沒有結婚的真意，但為了使白雪蘭得以結婚方式在臺灣地區工作，兩人在泰國辦理假結婚手續，取得不實的結婚證明書等資料。白秋霖於93年11月18日持這些文件前往臺中縣沙鹿鎮戶政事務所，申請辦理結婚戶籍登記，使承辦人員將虛偽辦理結婚的不實事項登載於職務上掌管的戶籍登記資料電子資訊檔案紀錄和戶籍謄本等公文書，損害戶政機關對於戶籍登記管理的正確性。

在94年3月30日，兩人向法院訴請離婚，經法院判決離婚後，白秋霖於94年7月7日再次前往戶政事務所辦理離婚登記，使承辦人員將虛偽辦理離婚的不實事項登載於戶籍登記資料。最後，白雪蘭因逾期居留於99年4月6日被查獲。

判決結果：被告白秋霖與白雪蘭達成協商合意，被告願受有期徒刑6月，減為有期徒刑3月，緩刑2年之宣告。
Judgement 2:
Origin Length: 7083
Split Judgement...
First Summary Length: 1818
First Summary Context:  以下是根據您提供的法律判決書內容所做的事實摘要：

判決書內容摘要：
一、被告乙○○明知其兄甲○○於2014年擔任屏東縣恆春鎮公所清潔隊代理隊長，但恆春鎮清潔隊徵才公告要求錄取人員不得與機關首長或長官有三親等以內的親屬關係。乙○○仍參加徵選，通過面試和體能測驗，並被列為正取。為使公務員登載不實，乙○○出具與首長或主管長官無三親等親屬關係的具結書，並提供相關證件。恆春鎮公所承辦人員丁○○和戊○○形式上審核後，誤以為乙○○符合進用條件，登載乙○○為恆春鎮清潔隊員，表彰其符合進用條件。
二、案件經法務部廉政署移送臺灣屏

In [32]:
import json

processed_data = []

for data in train_data[:50]:
    judgement = data['fact']
    processed_data.append(process_judgement(judgement))

with open("summary_output/processed_data.json", "w", encoding="utf-8") as json_file:
    json.dump(processed_data, json_file, ensure_ascii=False, indent=4)

Origin Length: 6546
Judgement processing...
First summary complete!
Scenario generating...
Scenario complete!
Origin Length: 3385
Judgement processing...
First summary complete!
Scenario generating...
Scenario complete!
Origin Length: 1331
Scenario generating...
Scenario complete!
Origin Length: 10172
Judgement processing...
First summary complete!
Scenario generating...
Scenario complete!
Origin Length: 9266
Judgement processing...
First summary complete!
Scenario generating...
Scenario complete!
Origin Length: 13669
Judgement processing...
First summary complete!
Scenario generating...
Scenario complete!
Origin Length: 328
Scenario generating...
Scenario complete!
Origin Length: 7083
Judgement processing...
First summary complete!
Scenario generating...
Scenario complete!
Origin Length: 8806
Judgement processing...
First summary complete!
Scenario generating...
Scenario complete!
Origin Length: 12726
Judgement processing...
First summary complete!
Scenario generating...
Scenario comp

# 10k-15k

In [62]:
# 長度12726
judgement = train_data[9]['fact']
print(f'Origin Length: {len(judgement)}')

if len(judgement) > 3000:
    fact_summary = split_passage(judgement)

print(f'Length: {len(fact_summary)}')
print(f'First Summayr: {fact_summary}')
print(f'='*100)
scenario_with_example = with_example(fact_summary)
print(scenario_with_example)
scenario_without_example = without_example(fact_summary)
print(scenario_without_example)

Origin Length: 12726
Length: 2958
First Summayr:  范姜慧雯為胡毓銘之妻，胡毓銘於桃園市中壢區開設寰東動物醫院，黃莉於96年至97年間在寰東動物醫院內擔任獸醫，並於97年間離職，後自行於桃園市楊梅區開設東昇動物醫院。何炳松於98年3月間向胡毓銘頂讓寰東動物醫院，將該動物醫院改名為環東動物醫院，由何炳松擔任環東動物醫院之負責人及獸醫，范姜慧雯則與何炳松合作，負責辦理環東動物醫院內之犬貓出口報關作業。范姜慧雯明知依動物傳染病防治條例第36條規定，輸入國需要輸出檢疫證明文件者，須申請檢疫，並檢附檢疫證明文書，始得自臺灣輸出動物，竟分別為下列行為：

1. 基於行使偽造私文書之犯意，未得黃莉之同意，於不詳時間，在住處內，以筆記型電腦偽造如附表一「偽造文書名稱」欄所示之文件，並委託不知情之刻印店人員，偽刻「獸醫師黃莉」、「東昇動物醫院」之印章各1個，嗣分別於如附表一「偽造日期」欄所示日期，在臺灣桃園國際機場內，於前開文件上偽簽黃莉之英文姓名「HuangLi」，並持前開偽造之「獸醫師黃莉」、「東昇動物醫院」印章，偽造如附表一「應沒收之物」欄所示之印文，藉此表彰前開文件均係黃莉親自執行狂犬病免疫注射並開立，再將前開文件透過不知情之魏明鋒、汎樺有限公司等報關行送交行政院農業委員會動植物防疫檢疫局新竹分局（下稱防疫局）審核而行使之，經防疫局審核前開文件通過，並核發輸出動物產品檢疫證明書後，魏明鋒及汎樺有限公司等報關行再將犬隻輸出國外，均足生損害於黃莉及防疫局審核文件及管理動物輸出資料之正確性。
2. 復另基於行使偽造私文書及業務登載不實文書之犯意，未得何炳松之同意，於不詳時間，在住處內，以筆記型電腦偽造如附表二「偽造文書名稱」欄所示之文件，並委託不知情之刻印店人員，偽刻「獸醫師何炳松」、「環東動物醫院」之印章各1個，嗣分別於附表二編號(一)、(四)至(十六)「偽造日期」欄所示日期，在附表二編號(一)「輸出犬貓免疫注射證明書」、編號(四)至(十六)之「偽造文書名稱」欄之文件上偽簽何炳松之英文姓名「H.P.SONG」，並持前開偽刻之「獸醫師何炳松」、「環東動物醫院」印章，偽造如附表二編號(一)「輸出犬貓免疫注射證明書」、編號(四)至(十六)「應沒收之物」欄所示之印文，藉此表彰前開文件均係何炳松親自執行狂犬病免疫注射並開立，且明知如附表二編號

# 做情節摘要(看有沒有CoT有什麼差)

In [ ]:
# 教他怎麼做情節摘要，測有沒有CoT的能力
# query是這次的input
# query = ''
CoT_prompt = f'我會給你三個範例 Example 1, Example 2 and Example 3 (輸入:Text,輸出:Context Summary) Example1 : Text輸入:孫桂蘭與孟奇於民國83年7月10日結婚,孫桂蘭係孟奇之子女即孟慶明、孟慶虹、孟慶經、孟慶緯、孟蓉之繼母。孟奇於96年1月22日死亡,孫桂蘭明知孟奇死亡後,其名下財產均為遺產,於遺產分割前屬孫桂蘭、孟慶明、孟慶虹、孟慶經、孟慶緯、孟蓉全體繼承人公同共有,未經全體繼承人同意,任一繼承人不得擅自處分,竟意圖為自己不法所有,基於詐欺取財及行使偽造私文書之犯意,於附表各編號所示之時間,至附表各編號所示金融機構,冒用孟奇之名義,在空白取款憑證填寫如附表各編號所示之日期、帳號及金額,並於「取款印鑑」欄,使用孟奇之印章盜蓋印文1枚,表示係孟奇欲自附表各編號所示金融機構帳戶內提領附表各編號所示提款金額,而偽造孟奇名義之取款憑證私文書後,再連同上開帳戶存摺持之向不知情之附表各編號所示金融機構承辦人員辦理取款而行使之,致該金融機構承辦人員陷於錯誤,誤認孫桂蘭係經孟奇本人授權而為上開手續,如數交付如附表各編號所示金額,均足以生損害於孟慶明、孟慶虹、孟慶經、孟慶緯(已歿)、孟蓉之權益及日盛國際商業銀行股份有限公司《下稱日盛商業銀行》、合作金庫商業銀行股份有限公司竹塹分行《下稱合作金庫商業銀行竹塹分行》、中華郵政股份有限公司清華大學郵局《下稱清華大學郵局》、永豐商業銀行股份有限公司《下稱永豐商業銀行》等金融機構對於孟奇帳戶資料、存款管理之正確性。 Context Summary輸出:在這個案件中,涉及到一段婚姻關係,並有繼承人之間的財產分割。一名已故的個人的遺產成為了糾紛的焦點。在其去世後,其配偶及子女成為了繼承人。然而,配偶明知其去世,卻企圖以詐欺和偽造私文書的方式來獲取他們的共同財產。她冒用已故人的名義,填寫了取款憑證並偽造了他的印章,使金融機構誤以為是合法的取款要求。最終,金融機構執行了這些虛偽的指示,導致繼承人之間的權益受損,也對涉及的金融機構造成了損害。这些行為涉及偽造私文書、詐欺取財,並對繼承人和金融機構產生了不良影響。 Example 2 : Text輸入:丁○○○為陳全清之妻,己○○為陳全清之弟即甲○○之兒媳,乙○○、陳全清、甲○○、廖陳專、丙○○、陳素美、陳月娥(下稱乙○○等7人)均為被繼承人陳秋火之子女,緣陳秋火於民國107年1月27日死亡,乙○○等7人均為陳秋火之法定繼承人。丁○○○、己○○明知陳秋火業已死亡,無法再以其名義為任何法律行為,且陳秋火所遺留之存款由全體繼承人繼承,應以全體繼承人名義具領款項,或全體繼承人同意委任代理人,依據繼承之程序,始得提領款項,丁○○○、己○○竟未得乙○○之同意,共同基於行使偽造私文書之犯意聯絡,將其等保管之陳秋火之存摺、印章取出,於107年1月29日9時55分許,共同前往位於苗栗縣○○鎮○○路0號之玉山商業銀行股份有限公司(下稱玉山銀行)後龍分行,冒用陳秋火之名義,申辦:將陳秋火之玉山銀行後龍分行定期存款(帳號依序分別為000000000號、000000000號、000000000號,金額均為新臺幣(下同)10萬元,共計30萬元)解約,存入陳秋火之同銀行帳號為0000000000000號帳戶(下稱甲帳戶)後,甲帳戶餘額為102萬2325元,再自甲帳戶提領現金25萬元,並將餘額77萬2325元匯至不知情之陳永聰(為甲○○之子、己○○之夫,另經臺灣苗栗地方檢察署檢察官為不起訴處分)之同銀行帳號0000000000000號帳戶(下稱乙帳戶)等交易,復持上開存摺、印章交由不知情之玉山銀行後龍分行承辦人員盜蓋於內部傳票、取款憑條上(偽造之文件名稱、交易摘要及金額、欄位、盜蓋印章之印文及數量均詳如附表所載),以利用該承辦人員偽造屬私文書之上開內部傳票、取款憑條後,據以交由該承辦人員辦理而行使之,致使承辦之玉山銀行人員誤認陳秋火尚在世並授權,而將上開陳秋火之定期存款解約及存入甲帳戶內之存款102萬2325萬元憑摺支取,除其中25萬元提領現金外,餘77萬2325元匯入乙帳戶,足以生損害於陳秋火之繼承人及玉山銀行後龍分行對於存款帳戶管理之正確性。Context Summary輸出:這宗案件涉及家族關係與遺產問題。已故個人的遺產引發了爭議。他的配偶與子女是合法繼承人,但配偶卻企圖透過詐欺手段來侵占共同財產。她冒用已故人的身份,偽造文件並盗用印章,讓金融機構誤以為是合法的交易。最終,金融機構執行了這些虛偽指示,導致繼承人的權益受損,也給金融機構造成損失。這些行為牽涉到偽造文件和詐騙,對繼承人和金融機構都帶來不良影響。Example 3: Text輸入:吳文錦係吳清海(民國108年2月2日死亡)之長子,吳芸安則係吳清海次子吳文樑(於103年3月22日死亡)之女,為吳文樑之代位繼承人,吳淑妮、吳金桂、吳淑惠、吳月嬌亦為吳清海之子女,惟吳文錦於吳清海過世後,為辦理吳清海之後事相關事宜時,明知不得再以吳清海之名義提領存款,須經全體繼承人同意或授權,始得提領存款,竟未經全體繼承人之同意或授權,基於行使偽造私文書之犯意,接續於108年2月11日、108年2月20日,持其所保管吳清海之雲林縣○○鄉○○0○號:000-0000000000000號,下稱四湖鄉農會帳戶)之存摺、印鑑,前往雲林縣○○鄉○○村○○○路00號之雲林縣四湖鄉農會,冒用吳清海之名義,在該農會之取款憑條存戶簽章欄上,填寫領取新臺幣(下同)337,600元、7,290元,並偽簽「吳清海」之署押1枚,盜蓋「吳清海」印文2枚(詳如附表所示),而偽造吳清海同意提領337,600元、7,290元之私文書,並持該偽造之取款憑條交予上開農會之不知情承辦人員而行使之,使吳文錦自前開帳戶提領共344,890元,足生損害於吳清海全體繼承人及四湖鄉農會對客戶存款管理之正確性。Context Summary輸出:這宗案件涉及家族關係和遺產問題。已故個人的子女是合法繼承人,但其中一人明知不得未經全體繼承人同意或授權就提領存款。他使用偽造文件,包括存摺和印鑑,前往當地農會冒充已故父親的身份,提領款項,造成損害。这涉及偽造文件和可能的詐騙行為,對繼承人和農會的存款管理產生不良影響。 請你模仿Example 1 , Example 2, Example 3,來生成以下一個輸入(Text1)之context摘要,禁止出現人名。Text1 輸入:{query} Text1輸出:(請用繁體中文輸出)'

In [ ]:
# 不要做CoT，直接做摘要(會有人名)
summary_prompt = ' I will give you a description of the facts of a legal judgment, and help me summarize it into a scenario about 500 words without listing points.'

In [ ]:
# 判斷偽造文書構成要件：需為文書、有偽造行為、損害利益於他人
# jedgement是判決書內容
# jedgement = ''
element_prompt = f'我會給你一篇法律判決書內容，列出這篇判決書符合偽造文書構成要件：需為文書、有偽造行為、損害利益於他人的地方。判決書內容如下：{jedgement}'

In [2]:
import json

with open('dataset/all_data.json', 'r', encoding='utf-8') as f:
    all_data = json.load(f)

In [9]:
for idx, item in enumerate(all_data[:3]):
    judgement = item['fact']
    print(f'Number: {idx}')
    print(f'Length of Judgement: \n{len(judgement)}')
    print(f'Judgement: \n{judgement}')
    # 給他小費好像會比較好，可以多獎勵他
    element_prompt = f'你現在是一位專業的法官，我會給你一篇法律判決書內容，用列點列出這篇判決書分別符合以下三點偽造文書構成要件：一、偽造誰的什麼文書、二、有什麼偽造行為、三、損害利益於他人的地方。我會給你小費，請認真回答我的問題。判決書內容如下：{judgement}'
    # print(f'Prompt: \n{element_of_forgery}')
    element = get_breeze(element_prompt)
    print(f'Element: \n{element}')
    print(f'='*100)

Number: 0
Length of Judgement: 
7884
Judgement: 
一、曹鳳珠與林寶櫳前為夫妻，因曹鳳珠認為林寶櫳與曹丞君間有不正常的交往關係，2人因此感情生變，遂於民國（下同）99年3月1日在福建連江地方法院調解離婚。詎曹鳳珠為取得保護及教養子女之權利，竟基於行使偽造私文書之犯意，先於某不詳時間、地點，偽造曹丞君簽名及日期94年5月10日之切結書（內容為本人曹丞君因與有婦之夫林寶櫳不正常的密切通聯，被林寶櫳老婆查獲，進而告之本人老公陳葦丞，為求兩家息事寧人，本人在媽媽及姐姐見證下，於姐姐家簽下切結書給林寶櫳老婆曹鳳珠，並保證今後不再與林寶櫳來往，若再被抓到願依妨害家庭處理，此致曹鳳珠）1份，再於99年5月3日，向福建連江地方法院具狀提出而行使之，希冀以此方式取得對子女保護及教養之權利，而足生損害於曹丞君。嗣因林寶櫳於99年5月5日在福建連江地方法院閱卷時，發現上開切結書而向曹丞君詢問，經曹丞君否認後，始查悉上情。二、曹鳳珠因認定係曹丞君介入其與林寶櫳間之婚姻，導致2人離婚，而對曹丞君心懷怨懟，竟基於毀損他人名譽之犯意，先後分別於99年11月5日凌晨4時43分許、100年1月10日上午10時2分許，在其位於連江縣南竿鄉馬祖村85號住處，以SKYPE網路帳號tina900928號，經由網路家庭國際資訊股份有限公司(即PChomeOnline)簡訊系統及米瑟奇網路簡訊平台散佈內容各為「號外！議會之光，春光外洩，曹妓員於11月5日凌晨二點，在神農山莊的休旅車上，與神農山莊王經理幽會長達兩個半小時......。」（下稱簡訊1）、「議員之恥！馬祖之惡！坐領高薪不夠用，議員兼作雞員，曹妓員1月8日深夜三點直奔神農山莊作應召，服務的對象是林姓警員，警紀敗壞蕩然無存；乾爹乾哥變成表哥表弟，破壞別人幸福家庭妻離子散，十惡不赦！罪大惡極！！正義之士群起躂閥！！」（下稱簡訊2）等涉及曹丞君私德而不具公共利益關聯性之事，足以詆毀曹丞君名譽之簡訊。嗣曹丞君陸續接獲他人來電告知收到上開簡訊，始知悉上情。三、案經曹丞君分別告訴及訴由福建省連江縣警察局移送福建連江地方法院檢察署檢察官偵查起訴。理由一、按被告以外之人於審判外之陳述，雖不符刑事訴訟法第159條之1至第159條之4之規定，而經當事人於審判程序同意作為證據，法院審酌該言詞陳述或書面陳述作成時之情況，認為適當者